# Exemplo de como obter a RNA para emular FK no problema de Shalow Water 2D

In [ ]:
# Baixa o código KFS2d_rna (shallow water assimilado por FK e RNA) a partir do repositório espelho do github
!git clone https://github.com/robertopsouto/kfs2d_rna_mirror.git

In [ ]:
# Entra no diretório principal do código de shalow water
%cd kfs2d_rna_mirror

In [ ]:
# Compilando o código
!make

In [ ]:
# Roda o shell script que chama o executável compilado na célula anterior
# Usa filtro de Kalman para fazer a assimilação de dados a cada 10 passos no tempo.
# O ruído no dado de observação é de 10%
# Uso:
# ./run-KFS2d.sh assimType gridX gridY timeStep freqObsT freqObsX freqObsY percNoise neuronNumber

!./run-KFS2d.sh 1 10 10 100 10 2 2 0.1 10

In [ ]:
# Indo para o diretório com o dados que serão usados para treinar a RNA, por meio do MPCA
%cd output/training/

## Script `Octave` que normaliza os valores do modelo, da observação e da análise (obtida com a assimilação de dados). São gerados os arquivos de treinamento (`x.txt` e `y.txt`) de validação cruzada (`x_valid.txt` e `y_valid.txt`) e de generalização (`x_gen.txt` e `y_gen.txt`), que serão os dados de entrada para o `MPCA-ANN`.

In [ ]:
%--------------------------------------------------------------------------
% programa para normalizar os valores da variavel para usar no treinamento da RNA/MPCA
%--------------------------------------------------------------------------
clear all;
close all; clc;
%-----------------------
%leitura dos dados
%-----------------------
qM = load('qModelExpA.out');
qO = load('qObservExpA.out');
qA = load('qAnalysisExpA.out');

valNormInf=-1.0;
valNormSup=+1.0;

qM_norm=(max(qM)*valNormInf-min(qM)*valNormSup+qM*(valNormSup-valNormInf))/(max(qM)-min(qM));
qO_norm=(max(qO)*valNormInf-min(qO)*valNormSup+qO*(valNormSup-valNormInf))/(max(qO)-min(qO));
qA_norm=(max(qA)*valNormInf-min(qA)*valNormSup+qA*(valNormSup-valNormInf))/(max(qA)-min(qA));

[l,n]=size(qM);
pontos_por_ciclo=25;
ciclos=n/pontos_por_ciclo;

parc1 = floor(0.60*ciclos)*pontos_por_ciclo;
parc2 = ceil(0.10*ciclos)*pontos_por_ciclo;
parc3 = n - parc1 - parc2;

train_range=parc1;
valid_init=train_range+1;
valid_end=train_range+parc2;
gen_init=valid_end+1;
gen_end=valid_end+parc3;;

fileID = fopen('x.txt','w');
fprintf(fileID,'      %8.5f',qM_norm(1:train_range));
fprintf(fileID,'\n');
fclose(fileID);
fileID = fopen('x.txt','a');
fprintf(fileID,'      %8.5f',qO_norm(1:train_range));
fprintf(fileID,'\n');
fclose(fileID);
fileID = fopen('y.txt','w');
fprintf(fileID,'      %8.5f',qA_norm(1:train_range));
fprintf(fileID,'\n');
fclose(fileID);

fileID = fopen('x_valid.txt','w');
fprintf(fileID,'      %8.5f',qM_norm(valid_init:valid_end));
fprintf(fileID,'\n');
fclose(fileID);
fileID = fopen('x_valid.txt','a');
fprintf(fileID,'      %8.5f',qO_norm(valid_init:valid_end));
fprintf(fileID,'\n');
fclose(fileID);
fileID = fopen('y_valid.txt','w');
fprintf(fileID,'      %8.5f',qA_norm(valid_init:valid_end));
fprintf(fileID,'\n');
fclose(fileID);

fileID = fopen('x_gen.txt','w');
fprintf(fileID,'      %8.5f',qM_norm(gen_init:gen_end));
fprintf(fileID,'\n');
fclose(fileID);
fileID = fopen('x_gen.txt','a');
fprintf(fileID,'      %8.5f',qO_norm(gen_init:gen_end));
fprintf(fileID,'\n');
fclose(fileID);
fileID = fopen('y_gen.txt','w');
fprintf(fileID,'      %8.5f',qA_norm(gen_init:gen_end));
fprintf(fileID,'\n');
fclose(fileID);


In [ ]:
parc1

In [ ]:
parc2

In [ ]:
parc3

In [ ]:
!ls -ltr

In [ ]:
%cd ../../../

In [ ]:
# Baixa o código do mpac-ann do repositório github
!git clone https://github.com/scsr-inpe/mpca-ann.git

In [ ]:
# Entra no diretório principal do código
%cd mpca-ann

In [ ]:
# Alterna para um commit específico
!git checkout a52ad36

In [ ]:
# Remove os arquivos pré-compilados que são baixados
!make clean

In [ ]:
# Compila o código 
!make

In [ ]:
# Copia os arquivos de treinamento, de validação cruzada e de generalização gerados anteriormente no k, para o diretório 'data' do mpca-ann
!cp ../kfs2d_rna_mirror/output/training/*.txt data/

In [ ]:
# Entra no diretório config, para editar os arquivos de configuração do mpca-ann
%cd config/

In [ ]:
# Edita o arquivo original configuration.ini tal que:
#    NCLASSES seja igual ao número de pontos de treinamento em x.txt e y.txt (dado pela variável parc1) 
#    NCLASSESVALIDATION seja igual ao número de pontos de validação em x_valid.txt e y_valid.txt (dado pela variável parc2)
#    UPPER_HIDDEN_LAYERS=1 seja igual a LOW_HIDDEN_LAYERS=1, para forçar que a arquitetura da rede tenha somente uma camada
#    LOWER_FIRST_HIDDEN_LAYER=UPPER_FIRST_HIDDEN_LAYER=10, para forçar que esta camana tenha exatamente 10 neurônios
#    LOWER_ACTIVATION_FUNCTION=UPPER_ACTIVATION_FUNCTION=2, para forçar que se use com função de ativação a tangente hiperbólica

!sed -i -e 's/NCLASSES=2000/NCLASSES=150/g' \
        -e 's/NCLASSESVALIDATION=600/NCLASSESVALIDATION=25/g' \
        -e 's/UPPER_HIDDEN_LAYERS=2/UPPER_HIDDEN_LAYERS=1/g' \
        -e 's/LOWER_FIRST_HIDDEN_LAYER=5/LOWER_FIRST_HIDDEN_LAYER=10/g' \
        -e 's/UPPER_FIRST_HIDDEN_LAYER=25/UPPER_FIRST_HIDDEN_LAYER=10/g' \
        -e 's/LOWER_ACTIVATION_FUNCTION=1/LOWER_ACTIVATION_FUNCTION=2/g' \
        -e 's/UPPER_ACTIVATION_FUNCTION=3/UPPER_ACTIVATION_FUNCTION=2/g' \
        configuration.ini

In [ ]:
# Edita o arquivo original annConfig.ini, 
# para que o número de classes seja igual ao número de pontos generalização em x_gen.txt e y_gen.txt

!sed -i -e 's/600/75/g' annConfig.in

In [ ]:
# Retorna para o diretório do script MPCA-ANN

%cd ..

In [ ]:
# Roda e script com 1 Experimento, usando 4 Processos MPI

!./runMPCA 1 4

In [ ]:
# Roda esta etapa para obter a solução que possui o menor valor da função objetivo com relação aos dados de generalização

!./annMLP 1 4

In [ ]:
# Copia o arquivo ann#.best, relativo ao experimento que obteve no menor erro na etapa de generalização, para o diretório data

!cp output/ann1.best ../kfs2d_rna_mirror/data/ann.best

In [ ]:
# Entra no diretório data

%cd ../kfs2d_rna_mirror/data/

In [ ]:
# Roda o shell script que extrai a informação dos pesos e vies do arquivo ann.best
# São gerados 4 arquivos: wqcsExpA.dat  wqcoExpA.dat  bqcsExpA.dat  bqcoExpA.dat
# Estes 4 arquivos serão lidos pelo kfs2d, para emular o FK por RNA 

!./pesos_vies.sh ann.best

In [ ]:
!ls -ltr

In [ ]:
%cd ../

In [ ]:
# Roda o shell script que chama o executável compilado na célula anterior
# Usa RNA para emular FK na assimilação de dados, a cada 10 passos no tempo.
# Uso:
# ./run-KFS2d.sh assimType gridX gridY timeStep freqObsT freqObsX freqObsY percNoise neuronNumber

!./run-KFS2d.sh 2 10 10 100 10 2 2 0.1 10

In [ ]:
# Entra no diretório onde o resultado da análise obtido com a assimilação por RNA está salvo

%cd output/full/

## Script `Octave` que gera figura comparando as curvas do modelo (verdade), da análise obtida por assimilação com filtro de Kalman (FK), e também da análise obtida por assimilação com rede neural emulando filtro de Kalman (RNA).

In [ ]:
%--------------------------------------------------------------------------
% programa para plotar os graficos
%--------------------------------------------------------------------------
clear all;
close all; clc;
%-----------------------
%leitura dos dados
%-----------------------
qm = load('qModelExpA.out');
qa = load('qAnalysisExpA.out');
% dado da rede
qar = load('qAnalysisExpA_RNA.out');
%
ni = 10;
nj = ni;
nk = 100;
%
ninj = ni*nj;
p = 7; % ponto
%
% dado do modelo
qm3D = reshape(qm,ni,nj,nk);
qpm10 = qm3D(p,p,:);
qpm10v = qpm10(:);
%
%valor estimado pelo Filtro de Kalman
qafk3D = reshape(qa,ni,nj,nk);
qpa10 = qafk3D(p,p,:);
qpa10v = qpa10(:);
%
%estimado pela Rede Neural
 qr3D = reshape(qar,ni,nj,nk);
 qpr10 = qr3D(p,p,:);
 qpr10v = qpr10(:);
%
figure(1)
plot(qpm10v,'b','linewidth',1.5); hold on;
plot(qpa10v,'r','linewidth',1.5);
plot(qpr10v,'g','linewidth',1.5);
title('variavel q'); grid on;
xlabel('tempo');
ylabel('q(7,7)');
legend('verdade','FK','RNA')
axis([1 100 -60 80])
%print -depsc variavelqExpA.eps;
print -dpng -r240 variavelqExpA.png;

In [ ]:
!ls -ltr

![Variavel q](kfs2d_rna_mirror/output/full/variavelqExpA.png)